In [1]:
import sys

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import s3fs
import scipy
import seaborn as sns
import statsmodels.api as sm
import statsmodels.formula.api as smf

sys.path.append("/Users/fgu/dev/projects/entropy")
import entropy.data.aggregators as ag
import entropy.data.cleaners as cl
import entropy.data.make_data as md
import entropy.data.selectors as sl
import entropy.data.validators as vl
import entropy.helpers.aws as ha
import entropy.helpers.data as hd
import entropy.helpers.helpers as hh

pd.set_option("display.max_rows", 120)
pd.set_option("display.max_columns", 120)
pd.set_option("max_colwidth", None)
%load_ext autoreload
%autoreload 2

fs = s3fs.S3FileSystem(profile="3di")

In [2]:
df = hd.read_txn_data("X77")
hd.inspect(df)

Time for read_txn_data                 : 27.40 seconds
(6,627,663, 21)


,date,user_id,amount,desc,merchant,tag_group,tag,account_id,account_last_refreshed,account_provider,account_type,debit,female,id,is_urban,latest_balance,logins,postcode,region_name,tag_auto,yob
0,2012-08-01,77,12.12,tv licence mbp - d/d,tv licensing,spend,communication,259583,2015-09-11,natwest bank,current,True,0.0,1212601,1.0,NaN,0.0,kt3 5,london,tv licence,1945.0
1,2012-08-01,77,13.81,amazon mktplace pmts amzn.com/billgbr,amazon,spend,services,259584,2014-03-06,natwest bank,credit card,True,0.0,1213850,1.0,NaN,0.0,kt3 5,london,enjoyment,1945.0


In [11]:
# Based on kantar data: https://www.kantarworldpanel.com/en/grocery-market-share/great-britain/snapshot

grocers = [
    "tesco",
    "sainsburys",
    "asda",
    "morrisons",
    "aldi",
    "co-op",
    "lidl",
    "waitrose",
    "iceland",
    "ocado",
]

In [8]:
def grocery_shop_entropy(df):
    df = df.copy()
    is_groc_spend = df.tag_auto.eq("food, groceries, household") & df.is_debit
    df["merchant"] = df.merchant.where(is_groc_spend, np.nan)
    counts = ag._entropy_counts(df, "merchant", wknd=True)
    return df.merchant.value_counts()
    return ag._entropy_scores(counts)


# sns.kdeplot(grocery_shop_entropy(df));
grocery_shop_entropy(df).head(50).index

CategoricalIndex(['tesco', 'sainsburys', 'co-op', 'marks and spencer', 'asda',
                  'waitrose', 'morrisons', 'aldi', 'lidl', 'poundland', 'spar',
                  'ocado', 'iceland', 'home bargains', 'one stop', 'mccolls',
                  'costco', 'nisa', 'scotmid co-operative', 'costcutter',
                  'londis', 'budgens', 'poundstretcher', 'eh booth & co',
                  'pound world', 'whole foods market', 'hello fresh',
                  'nespresso', 'riverford', 'abel & cole', 'farmfoods',
                  'heron foods', 'planet organic', 'best-one', 'amazon prime',
                  'muji', '99p stores', 'fresh and wild', 'john lewis',
                  'snax 24', 'dairy crest', 'af blakemore',
                  'beanbag natural health witney', 'philpotts', 'phones 4 u',
                  'phoenix life', 'phone and pay',
                  'philip williams and company', 'pixmania uk', 'photobox'],
                 categories=['1 & 1', '123-reg', '1rebel